In [36]:
# Sample Python code for user authorization
import os
import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import requests, re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")
import spacy
import en_core_web_sm
import en_core_web_lg
import en_vectors_web_lg
import en_core_web_md

from dateparser.search import search_dates
import arrow
import datetime
import dateutil.parser
import csv


In [37]:
# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret2.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [38]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=802025607717-0mj0rii4j0s7hiki3b34n7t8iue1amkg.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=mUephZ5F881Z6GtfCJu0hDC3eJHdgk&prompt=consent&access_type=offline
Enter the authorization code: 4/5QDDcFEkr7Dm1-vPlrqT4AFFhnvdUotZAnHP-DqeTzmkXr_9ifR7ZAg


## Initialize Variables from Codebook

In [39]:
# Get the channel title from pdf on the column "Channel Name"
channel_title='BuzzFeedVideo'
influencer_name='Try Guys'

In [40]:
# Get the series title from pdf on the column "Series Name"
series = ''    

In [66]:
# Input the filename 
#csv_output = "batch86_sellner.csv"
# batch38_paulwatson.csv
csv_output = "tryguys_pants.csv"

In [49]:
playlist_id = 'PLCrHLhC-zBIoWPh3HgoZwoDdqYWluSA0Z'

## Get Channel Playlists

In [50]:
# channel_title='The Daily Wire'

In [51]:
# result = service.search().list(\
# q=channel_title,part='snippet',
# maxResults=5).execute()


In [52]:
def get_channel_id (channel_title='The Daily Wire'):
    result = service.search().list(
        q=channel_title,part='snippet',
        maxResults=5).execute()
    for res in result['items']:
        if(res['snippet']['title']==channel_title):
            print(res['id']['channelId'])
            channel_id = res['id']['channelId'] 
            break
    return channel_id
    

In [53]:
def transform_date(input_date_str,new_timezone=new_timezone):
    video_date_posted_iso = dateutil.parser.parse(input_date_str).astimezone(new_timezone)
    video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
    return video_date_posted,video_date_posted_iso
def transform_date_custom(input_date_str,new_timezone=new_timezone):
    video_date_posted_iso = dateutil.parser.parse(input_date_str).astimezone(new_timezone)
    video_date_posted = video_date_posted_iso.strftime("%Y_%m_%d_%H_%M_%S")
    return video_date_posted
def transform_date_iso(input_date_str,new_timezone=new_timezone):
    video_date_posted_iso = dateutil.parser.parse(input_date_str).astimezone(new_timezone)
    return video_date_posted_iso

In [54]:
def set_flags (flag):
    video=0
    comment=0
    reply=0
    if flag=='video':
        video=1
    elif flag=='comment':
        comment=1
    elif flag=='reply':
        reply=1
    return video,comment,reply

## End-to-end Playlist to Comments

In [67]:
comment_op=''
reply_op=''
transcript=''
date_scrape = datetime.datetime.now(new_timezone).strftime("%Y_%m_%d_%H_%M_%S")       
cutoff_date = datetime.datetime(2018, 12, 2, 0, 0, 0, tzinfo=new_timezone)
nextPageToken=''
max_results=50
# csv_output = 'batch3_warski.csv'
#  delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['index', 'video_id','video_url',
                 'video_title',"channel",'series','video_snippet',
                 'video','comment','reply','video_op',
                 'comment_op','reply_op','date_posted',
                 'content','date_scrape']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
#     for i in range(int(video_count/max_results) +1):
    while True:
        sleep(1)
        try:
            result_vids = service.playlistItems().list(
                playlistId=playlist_id,part='snippet,contentDetails',
                maxResults=max_results,pageToken=nextPageToken).execute()
        except:
            break
        for result_item in result_vids['items'] :
            try:    
                video_date_posted = result_item['contentDetails']['videoPublishedAt']
            except:
                video_date_posted = result_item['snippet']['publishedAt']

            video_date_posted_iso = dateutil.parser.parse(video_date_posted).astimezone(new_timezone)
            if(cutoff_date<video_date_posted_iso):
                continue
            video_date_posted = transform_date_custom(video_date_posted)
            video_id = result_item['contentDetails']['videoId']
            #     video_index = 
            video_index = result_item['snippet']['position']
            video_title = result_item['snippet']['title']
            video_snippet = result_item['snippet']['description']    
            video_op =result_item['snippet']['channelTitle']
            my_url='https://www.youtube.com/watch?v=%s' %(video_id)
#             vid_idx=i*max_results + int(video_index)
            display(video_title,video_index)
            
            video,comment,reply=set_flags(flag='video')
            x =  { "index": video_id,"video_id": video_id, "video_url":my_url,
                "video_title":video_title, "channel":channel_title,
                "series":series, "video_snippet":video_snippet,
                "video":video,"comment":comment,"reply":reply,
                "video_op":influencer_name, "comment_op":comment_op,
                "reply_op":reply_op,"date_posted":video_date_posted,
                "content":transcript,"date_scrape":date_scrape}   
            writer.writerow(x)
            
# START COMMENT EXTRACTION
            comment_nextPageToken = ''
            while True:
                try:
                    result_comments= service.commentThreads().list(
                        part="snippet,replies",
                        videoId=video_id,
                        textFormat="plainText",
                        maxResults=100,
                        pageToken = comment_nextPageToken
                      ).execute()
                except:
                    break
                    
                for comment_ind in result_comments['items']:
                    try:
                        reply_op=''
                        comment_op = comment_ind['snippet']['topLevelComment']['snippet']['authorDisplayName']
    #                     display(comment_op)
                        comment_date_posted = comment_ind['snippet']['topLevelComment']['snippet']['publishedAt']
                    #     display(comment_ind['snippet']['topLevelComment']['snippet']['publishedAt'])
                        comment_date_posted=transform_date_custom(comment_date_posted)
                        comment_index = comment_ind['snippet']['topLevelComment']['id']
                        comment_content = comment_ind['snippet']['topLevelComment']['snippet']['textOriginal']
                        video,comment,reply=set_flags(flag='comment')
                #         x =  { "video_id": video_id, 
                #             "video":video,"comment":comment,"reply":reply,
                #             "comment_op":comment_op,
                #             "reply_op":reply_op,"date_posted":comment_date_posted,
                #             "content":comment_content,"date_scrape":date_scrape}  
                        x =  { "index": comment_index,"video_id": video_id, "video_url":my_url,
                            "video_title":video_title, "channel":channel_title,
                            "series":series, "video_snippet":video_snippet,
                            "video":video,"comment":comment,"reply":reply,
                            "video_op":influencer_name,"comment_op":comment_op,
                            "reply_op":reply_op,"date_posted":comment_date_posted,
                            "content":comment_content,"date_scrape":date_scrape}          

                        writer.writerow(x)      

                    except:
                        continue
                    if comment_ind['snippet']['totalReplyCount']>0:
#                         display('True')
#                         display(comment_ind['snippet']['totalReplyCount'])
                        try:
                            for reply_result in comment_ind['replies']['comments']:
                                reply_index=reply_result['id']
                                reply_op = reply_result['snippet']['authorDisplayName']
                                reply_content = reply_result['snippet']['textOriginal']
                                reply_date_posted = transform_date_custom(reply_result['snippet']['publishedAt'])
                                video,comment,reply=set_flags(flag='reply')
                #                 x =  { "video_id": video_id, 
                #                     "video":video,"comment":comment,"reply":reply,
                #                     "comment_op":comment_op,
                #                     "reply_op":reply_op,"date_posted":reply_date_posted,
                #                     "content":reply_content,"date_scrape":date_scrape}  
                #                 display(x)   
                                x =  {"index": reply_index,"video_id": video_id, "video_url":my_url,
                                    "video_title":video_title, "channel":channel_title,
                                    "series":series, "video_snippet":video_snippet,
                                    "video":video,"comment":comment,"reply":reply,
                                    "video_op":influencer_name, "comment_op":comment_op,
                                    "reply_op":reply_op,"date_posted":reply_date_posted,
                                    "content":reply_content,"date_scrape":date_scrape} 
                                writer.writerow(x)      
                        except:
                            pass

                try:
                    comment_nextPageToken = result_comments['nextPageToken']
                except:
                    break
            
        try:
            nextPageToken = result_vids['nextPageToken']
            
        except:
            break
            print('no more vids')

'The Try Guys Wear Women’s Pants (feat. Safiya Nygaard)'

0